In [1]:
import json
from pathlib import Path

In [2]:
RED = '\x1b[31m'
BLUE = '\x1b[34m'
NC = '\x1b[0m'

In [7]:
dataset_dir = Path('../data/multiwoz2_parsed')

raw_dials_path = dataset_dir / '..' / 'MULTIWOZ2 2' / 'data.json'
delex_dials_path = dataset_dir / 'multi-woz' / 'delex.json'
train_dials_path = dataset_dir / 'train_dials.json'
valid_dials_path = dataset_dir / 'val_dials.json'
test_dials_path = dataset_dir / 'test_dials.json'

gen_dir = Path('multiwoz/model/data')

valid_dials_gen_path = gen_dir / 'val_dials' / 'val_dials_gen.json'
test_dials_gen_path = gen_dir / 'test_dials' / 'test_dials_gen.json'

In [8]:
with open(raw_dials_path, 'r') as raw_dial_f:
    raw_dials = json.load(raw_dial_f)
with open(delex_dials_path, 'r') as delex_dial_f:
    delex_dials = json.load(delex_dial_f)
with open(valid_dials_path, 'r') as val_dial_f:
    valid_dials = json.load(val_dial_f)
with open(test_dials_path, 'r') as test_dial_f:
    test_dials = json.load(test_dial_f)

with open(valid_dials_gen_path, 'r') as val_dial_gen_f:
    valid_dials_gen = json.load(val_dial_gen_f)
with open(test_dials_gen_path, 'r') as test_dial_gen_f:
    test_dials_gen = json.load(test_dial_gen_f)

In [9]:
def show_turn(dial_id, turn_id, filt='11111'):
    return '\n'.join(filter(None, [
        '' if filt[0] == '0' else '{}User   (raw):\n{}\n{}'.
            format(RED, raw_dials[dial_id]['log'][turn_id*2]['text'], NC),
        '' if filt[1] == '0' else '{}User   (delex) (input):\n{}\n{}'.
            format(RED, valid_dials[dial_id]['usr'][turn_id].strip(), NC),
        '' if filt[2] == '0' else '{}System (raw):\n{}\n{}'.
            format(BLUE, raw_dials[dial_id]['log'][turn_id*2+1]['text'], NC),
        '' if filt[3] == '0' else '{}System (delex) (ground truth):\n{}\n{}'.
            format(BLUE, valid_dials[dial_id]['sys'][turn_id].strip(), NC),
        '' if filt[4] == '0' else '{}System (gen):\n{}\n{}'.
            format(BLUE, valid_dials_gen[dial_id][turn_id], NC),
    ]))

In [12]:
list(valid_dials.keys())[:10]

['PMUL3233.json',
 'SNG01627.json',
 'PMUL1402.json',
 'MUL0242.json',
 'PMUL1072.json',
 'PMUL3048.json',
 'PMUL0958.json',
 'PMUL3979.json',
 'MUL1409.json',
 'PMUL4828.json']

In [47]:
single_dial_ids = []
mul_dial_ids = []

for dial_id in raw_dials.keys():
    if 'SNG' in dial_id or 'WOZ' in dial_id:
        single_dial_ids.append(dial_id)
    elif 'MUL' in dial_id:
        mul_dial_ids.append(dial_id)
    else:
        assert False, dial_id

In [50]:
dial_id = single_dial_ids[25]  # 'MUL2105.json'
# dial = valid_dials[dial_id]
n_turns = len(raw_dials[dial_id]['log']) // 2

print(dial_id)
for turn_id in range(n_turns):
    print(show_turn(dial_id, turn_id, filt='10100'))

SNG0285.json
User   (raw):
I need a train that leaves on Sunday for Broxbourne.

System (raw):
I have 19 trains going to broxbourne on Sunday. To help narrow it down, where would you like to depart from and what time?

User   (raw):
I want it to leave from cambridge after 09:15.

System (raw):
Let me look up those times for you. One minute.

User   (raw):
Not looking to book, can I just get the train ID, price, and travel time? Thanks!

System (raw):
I'm sorry, I'm not finding that fits those criteria. Is there a different time you could leave? 

User   (raw):
There are no trains going to broxbourne from cambridge on Sunday after 09:15?

System (raw):
No, I'm sorry, it doesn't appear that those trains run on Sundays.  Is there a different day you could travel?  Also, will you leave from Broxbourne or Cambridge?

User   (raw):
I need to go TO Broxbourne FROM Cambridge on Sunday, leaving any time after 9:15. Please check again with those details, please. 

System (raw):
I'm terribly sorr

In [51]:
raw_dials[dial_id]['log']

[{'metadata': {},
  'text': 'I need a train that leaves on Sunday for Broxbourne.'},
 {'metadata': {'attraction': {'book': {'booked': []},
    'semi': {'area': '', 'name': '', 'type': ''}},
   'hospital': {'book': {'booked': []}, 'semi': {'department': ''}},
   'hotel': {'book': {'booked': [], 'day': '', 'people': '', 'stay': ''},
    'semi': {'area': '',
     'internet': '',
     'name': '',
     'parking': '',
     'pricerange': '',
     'stars': '',
     'type': ''}},
   'police': {'book': {'booked': []}, 'semi': {}},
   'restaurant': {'book': {'booked': [], 'day': '', 'people': '', 'time': ''},
    'semi': {'area': '', 'food': '', 'name': '', 'pricerange': ''}},
   'taxi': {'book': {'booked': []},
    'semi': {'arriveBy': '',
     'departure': '',
     'destination': '',
     'leaveAt': ''}},
   'train': {'book': {'booked': [], 'people': ''},
    'semi': {'arriveBy': 'not mentioned',
     'day': 'sunday',
     'departure': 'not mentioned',
     'destination': 'broxbourne',
     'le

In [15]:
dial.keys()

dict_keys(['sys', 'db', 'usr', 'bs'])

In [16]:
dial['sys']

['congratulations on your upcoming anniversary ! [value_place] offers a variety of lodging options , what is your price range ?',
 'i have several options for you , is there a particular area you are interested in during your stay ?',
 'i have [value_count] guesthouse that fits that criteria , [attraction_name] . would you like me to book for you ?',
 '      ok , your hotel stay at [hotel_name] is booked , reference number [hotel_reference] . will there be anything else ?      ',
 '      sure , it s located in the [value_area] area of town . the phone number is [attraction_phone] . the entrance fee is [value_count] pounds .      ',
 '      the address is [attraction_address] . did you want any more information about the museum ?      ',
 '      the entrance fee is [value_count] pounds .      ',
 '      and what is your destination ?      ',
 '            your taxi has been booked . it will be a [taxi_type] and the contact number is [taxi_phone] .            ',
 '            have a nice

In [17]:
dial['usr']

['my husband and i are celebrating our anniversary and want to find a great place to stay in town .',
 'i would like a [value_count] star guesthouse that include -s free parking .',
 'yes should be in the [value_area]',
 "yes , please ! we'll arrive on [value_day] and stay [value_count] nights . just the [value_count] of us , of course !",
 'i am wanting to know more about the [attraction_name] .',
 'what is the address , please .',
 'yes , i need the entrance fee . i will also need a taxi to pick me up by [value_time] . i need the contact number and car type please .',
 'can you book the taxi for me at [value_time] leaving from the museum please . i will need the contact number and the car type .',
 'i will leave the [value_place] museum at [value_time] , going to the the [hotel_name] .',
 'thanks so much . that is all i need for today . bye .']

In [18]:
len(dial['bs']), len(dial['bs'][0])

(10, 94)

In [19]:
len(dial['db']), len(dial['db'][0])

(10, 30)

In [20]:
def decode_db(v):
    assert len(v) == 30, len(v)
    domains = ['restaurant', 'hotel', 'attraction', 'train']
    for i, d in enumerate(domains):
        print('{}: {}'.format(d, v[i*6: i*6+6]))
    print('restaurant booking: {}'.format(v[24: 26]))
    print('hotel booking: {}'.format(v[26: 28]))
    print('train booking: {}'.format(v[28:]))

In [21]:
for i, db_vector in enumerate(dial['db']):
    print('# Turn {}'.format(i))
    decode_db(db_vector)

# Turn 0
restaurant: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
hotel: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
attraction: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
train: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
restaurant booking: [1.0, 0.0]
hotel booking: [1.0, 0.0]
train booking: [1.0, 0.0]
# Turn 1
restaurant: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
hotel: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
attraction: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
train: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
restaurant booking: [1.0, 0.0]
hotel booking: [1.0, 0.0]
train booking: [1.0, 0.0]
# Turn 2
restaurant: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
hotel: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
attraction: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
train: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
restaurant booking: [1.0, 0.0]
hotel booking: [1.0, 0.0]
train booking: [1.0, 0.0]
# Turn 3
restaurant: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
hotel: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
attraction: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
train: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
restaurant booking: [1.0, 0.0]
hotel booking: [0.0, 1.0]
train book